# Parallelism on the JVM II

The synchronized block is an example of a synchronization primitive. A construct that allows threads to exchange information. Importantly, multiple synchronized blocks can compose.

Invocations of the synchronized block can nest.

In [1]:
class Account(private var amount: Int = 0) {
def transfer(target: Account, n: Int) =
this.synchronized {
target.synchronized {
this.amount -= n
target.amount += n
}
}
}

defined class Account

`Account` class has method `transfer` which is used to transfer some amount to target account. Many invocations of transfer could occur in parallel. 

To implement transfer method, you donot want to have single global synchronized block. This is bottleneck for parallelism, especially many transfers occur in parallel. 

Instead we use more fine grained synchronization, specifically we use synchronized block on source and target account, this will make sure code in nested block is executed atomic, both for the threads using the source account, `this`, and threads using the target account.

For example a thread running the transfer method, first obtains the monitor on the account `A1` and then account `A2` and then transfer the money. In parallel another thread can do same thing for accounts `A3` and `A4`.

Let's see a demo.

In [3]:
def startThread(a: Account, b: Account, n: Int)={
    val t = new Thread{
        override def run() = {
            for(i <- 0 until n ){
                a.transfer(b, 1)
            }
        }
    }
    t.start()
    t
}

defined function startThread

In [4]:
val a1 = new Account(50000)
val a2 = new Account(70000)

val t = startThread(a1, a2, 15000)
val s = startThread(a2, a1, 15000)

a1: Account = ammonite.$sess.cmd0$Helper$Account@142980e
a2: Account = ammonite.$sess.cmd0$Helper$Account@102e0f7
t: Thread = Thread[Thread-0,5,main]
s: Thread = Thread[Thread-1,5,main]

In [ ]:
t.join()
s.join()

This program never completes, because thread `t` and `s` never completes so join blocks the main thread.

But Why?

> **Deadlock** is a scenario in which two or more threads compete for resources (such as monitor ownership), and wait for each to finish without releasing the already acquired resources.

In our case, after having created two accounts

```scala

val a = new Account(50)
val b = new Account(70)

//thread T1
a.transfer(b,10)

//thread T2
b.transfer(a,10)

```

two seprate threads invoke transfer methods but in reverse order. In first case source account `a` and second case source account `b`. The two threads start the synchronized statement on these accounts in different order. The first thread enter on syncrhonized block `a` and second thread enter synchronized block on `b`. At this point, the first thread blocks untill monitor on object `b` becomes available and vice versa.

Neither thread makes progress so deadlock occurs.

How to resolve deadlocks?

One approach is to always acquire resources in the same order.

This assumes an ordering relationship on the resources.

In [6]:
private val x = new AnyRef {}
private var uidCount = 0L
def getUniqueId(): Long = x.synchronized{
uidCount = uidCount + 1
uidCount
}

class Account(private var amount: Int = 0) {

val uid = getUniqueId()
private def lockAndTransfer(target: Account, n: Int) =
this.synchronized {
target.synchronized {
this.amount -= n
target.amount += n
}
}    
def transfer(target: Account, n: Int) =
if (this.uid < target.uid) this.lockAndTransfer(target, n)
else target.lockAndTransfer(this, -n)
}



defined function getUniqueId
defined class Account

In case the target uid is larget than current account uid, negative amount is transfered, to the source account.

## Memory Model

Memory model is a set of rules that describes how threads interact when accessing shared memory.

Java Memory Model – the memory model for the JVM.

1. Two threads writing to separate locations in memory do not need synchronization.
2. A thread X that calls join on another thread Y is guaranteed to 
observe all the writes by thread Y after join returns.

The parallelism constructs in the remainder of the course are implemented in terms of:
* threads
* synchronization primitives such as synchronized

It is important to know what’s under the hood!